In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 13.4 kB/s eta 0:00:00a 0:00:14
-

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

In [ ]:
# using yfinance to extract stock data
tesla = yf.Ticker('TSLA')

In [ ]:
tesla_data = tesla.history(period='max')

In [ ]:
type(tesla_data)
tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
# use webscraping to extract Tesla revenue data

tesla_revenue_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm').text

In [ ]:
bs_soup = BeautifulSoup(tesla_revenue_data, 'html.parser')

In [ ]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

In [ ]:
for tr in bs_soup.find('tbody').find_all('tr'):
    td_list = tr.find_all('td')
    date = td_list[0].text
    revenu = td_list[1].text
    tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({'Date':[date] , 'Revenue':[revenu]})],ignore_index=True)

In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"",regex=True)

In [ ]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.tail()

In [ ]:
# using yfinance to extract stock data
gme = yf.Ticker('GME')

In [ ]:
gme_data = gme.history(period='max')

In [ ]:
gme_data.reset_index(inplace=True)

In [ ]:
gme_data.head()

In [ ]:
# Use Webscraping to Extract GME Revenue Data
gme_revenue_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html').text

In [ ]:
bs_soup_gme = BeautifulSoup(gme_revenue_data, 'html.parser')
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

In [ ]:
for tr in bs_soup_gme.find('tbody').find_all('tr'):
    td_list = tr.find_all('td')
    date = td_list[0].text
    revenu = td_list[1].text
    gme_revenue = pd.concat([gme_revenue,pd.DataFrame({'Date':[date] , 'Revenue':[revenu]})],ignore_index=True)

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"",regex=True)

In [ ]:
gme_revenue.tail()

In [ ]:
# Plot tesla Stock Graph
make_graph(tesla_data,tesla_revenue,'tesla')

In [ ]:
# Plot gme Stock Graph
make_graph(gme_data,gme_revenue,'GameStop')